In [ ]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import os

from google.colab import files

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive")

df = pd.read_csv('mbti_preprocessed.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### MBTI 분류 및 불용어 처리

In [ ]:
mbti_J = df[(df['type_label']==0) | (df['type_label']==2) | (df['type_label']==4) | (df['type_label']== 6) | (df['type_label']==8) | (df['type_label']==10) | (df['type_label']== 12) | (df['type_label']==14)]
mbti_P = df[(df['type_label']==1) | (df['type_label']==3) | (df['type_label']==5) | (df['type_label']== 7) | (df['type_label']==9) | (df['type_label']==11) | (df['type_label']== 13) | (df['type_label']==15)]

In [ ]:
from nltk.corpus import stopwords
stop = ['tapatalk', 'iphone', 'ha', 'using', 'enfjs', 'thats', 'kitteh', 'th', 'ya', 'alot', 'fucking', 'favourite', 'dont', 'sp', 'sx', 'dom', 'cant', 'esfjs', 'estps', 'enfps', 'entps', 'nf', 'sent', 'inferior', 'ive', 'score','particularly', 'youtube', 'husband', 'sensor', 'coffee', 'yea', 'idk', 'summer', 'fourm', 'welcome', 'much', 'istps', 'hahaha', 'wanna', 'istps', 'entjs', 'istjs', 'yep', 'typically', 'somebody', 'tmlt', 'mbti', 'type', 'intuition', 'typed', 'socionics','omg']
mbti_J['posts'] = mbti_J['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
mbti_P['posts'] = mbti_P['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### DTM 생성



In [ ]:
#MBTI_J
from sklearn.feature_extraction.text import CountVectorizer
vec_J = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    
document_term_matrix_J = vec_J.fit_transform(mbti_J['posts'])
DTM_J = pd.DataFrame(document_term_matrix_J.toarray(), columns=vec_J.get_feature_names())  

#MBTI_P
vec_P = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
document_term_matrix_P = vec_P.fit_transform(mbti_P['posts'])
DTM_P = pd.DataFrame(document_term_matrix_P.toarray(), columns=vec_P.get_feature_names())  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
transformer

MBTI_J_TFIDF = transformer.fit_transform(DTM_J)
print('#shape of tfidf matrix:', MBTI_J_TFIDF)

MBTI_P_TFIDF = transformer.fit_transform(DTM_P)
print('#shape of tfidf matrix:', MBTI_P_TFIDF)

#shape of tfidf matrix:   (0, 959)	0.12153565754715549
  (0, 951)	0.41610576294397184
  (0, 905)	0.1423330098332904
  (0, 881)	0.14504686368611694
  (0, 872)	0.1461476743456058
  (0, 858)	0.1344220990494861
  (0, 815)	0.12447315172173731
  (0, 794)	0.12484232114321542
  (0, 790)	0.14418983026731017
  (0, 759)	0.14173476276944358
  (0, 751)	0.14705322407062932
  (0, 690)	0.14728321656602572
  (0, 643)	0.1262292904032216
  (0, 559)	0.11977414627023433
  (0, 499)	0.12198984279200799
  (0, 487)	0.14966777727726757
  (0, 459)	0.12597310638179232
  (0, 454)	0.12086494802117395
  (0, 413)	0.12496622385786416
  (0, 408)	0.12279872409251294
  (0, 406)	0.12889671074748538
  (0, 398)	0.12176203378431663
  (0, 391)	0.14134138504995902
  (0, 389)	0.13458386877375791
  (0, 355)	0.14253465575598936
  :	:
  (3433, 362)	0.09931961219508986
  (3433, 354)	0.11813177894534815
  (3433, 277)	0.12129782669103495
  (3433, 261)	0.11949766738086107
  (3433, 259)	0.10371791020019089
  (3433, 258)	0.1014378888945

In [ ]:
MBTI_J_TFIDF_DF = pd.DataFrame(MBTI_J_TFIDF.toarray(), columns=vec_J.get_feature_names())
MBTI_J_TFIDF_DF

MBTI_P_TFIDF_DF = pd.DataFrame(MBTI_P_TFIDF.toarray(), columns=vec_P.get_feature_names())
MBTI_P_TFIDF_DF

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absolute,abstract,accepted,according,account,act like,acting,active,activity,adhd,...,would make,would never,would probably,would rather,would want,writer,yay,yes yes,yup,zone
0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.136620,0.000000
2,0.117966,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.111863,0.000000,0.0,0.0,0.0,0.113322,0.0,0.0,0.000000,0.000000
3,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.351351,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,0.095958,0.0,0.096227,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.108625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.171429
5237,0.000000,0.0,0.0,0.090892,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.097225,0.0,0.0,0.000000,0.000000
5238,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.122983,0.000000
5239,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.109412,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [ ]:
MBTI_J_TFIDF_SUM = MBTI_J_TFIDF_DF.sum()
MBTI_J_TFIDF_SUM.sort_values(ascending=False)[0:50]

girlfriend         39.824483
baby               39.198204
film               39.161330
gift               38.927940
psychology         38.356246
wife               38.008268
five               37.338024
gender             37.176703
wonderful          37.143090
law                36.672595
war                36.670822
dry                36.597571
rock               36.546442
nobody             36.522916
shocked            36.208536
pattern            36.095544
dude               36.088083
depressed          36.043122
facebook           35.896438
grade              35.848528
developed          35.803003
useful             35.604496
mbti type          35.556805
attractive         35.497909
upset              35.384235
religious          35.363178
close friend       35.276834
frustrating        35.239301
version            35.150402
lady               35.008644
stranger           34.954113
plus               34.891879
waiting            34.707464
tea                34.676437
handle        

In [ ]:
MBTI_P_TFIDF_SUM = MBTI_P_TFIDF_DF.sum() 
MBTI_P_TFIDF_SUM.sort_values(ascending=False)[0:50]

cognitive function    66.323729
band                  60.299862
sport                 60.222966
married               59.894600
gender                59.628617
series                58.569584
film                  58.245935
intuitive             57.268257
crush                 57.180006
five                  56.596141
disorder              56.422347
psychology            56.244087
nobody                56.196980
stranger              55.835960
letter                55.780979
scared                55.513442
star                  55.192767
student               55.185504
facebook              55.147135
helpful               55.058456
unhealthy             55.005013
typed                 54.548456
preference            54.391895
artist                54.382295
silly                 54.365232
played                54.231965
feeler                54.227603
approach              54.104981
article               53.977628
earth                 53.845645
business              53.845371
blushed 

### LDA 분석

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

document_term_matrix_J
print('CountVectorizer Shape:', document_term_matrix_J.shape)

lda = LatentDirichletAllocation(n_components=2)
lda.fit(document_term_matrix_J)

def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

feature_names = vec_J.get_feature_names()

display_topics(lda, feature_names, 10)

CountVectorizer Shape: (3434, 1000)
Topic # 0
pattern philosophy developed moral rational useful context claim religious thinker
Topic # 1
girlfriend film wonderful baby facebook grade gift city yesterday close friend


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
document_term_matrix_P
print('CountVectorizer Shape:', document_term_matrix_P.shape)

lda_P = LatentDirichletAllocation(n_components=2)
lda_P.fit(document_term_matrix_P)

feature_names = vec_P.get_feature_names()

display_topics(lda_P, feature_names, 10)

CountVectorizer Shape: (5241, 1000)
Topic # 0
band series film war philosophy played psychology star universe sport
Topic # 1
married helpful blushed unhealthy upset feeler scared cognitive function intuitive intense


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
